# Compare existing results

This sheet enables you to compare results that have already been generated. It includes functions to plot cell number in each generation for each condition, along with plots that indicate whether a combination of effects is greater than expected if they were additive.

In [ ]:
using Plots 
using CSV
using Tables
using DataFrames
plotly()

In [ ]:
maxGens=8 # maximum number of gens in total
global maxTimeTC=float(60*120) # how long should the entire simulation run

In [ ]:
#### Load patient data
patient_data = DataFrame(CSV.File("patientData.csv"))
patient_data = patient_data[completecases(patient_data), :]
patient_data = subset(patient_data, :TYPE => ByRow(==("CNA")))

In [ ]:
patients = unique(patient_data[!, :PATIENT])

In [ ]:
cA=palette(:seaborn_colorblind)

In [ ]:
#### Mut/Param data
muts2Params = DataFrame(CSV.File("muts2ParamsNEW.csv"))
# concatenate gene and alteration
muts2Params[!, "GeneAlt"] = string.(muts2Params[!, "GENE"], ":", muts2Params[!, "ALTERATION"])
## remove zeros
muts2Params = muts2Params[(muts2Params.PARAMETER .!= "0"), :];

In [ ]:
#### Load combined output data
CombOp = DataFrame(CSV.File("combinedOutput2_noDupes.csv"))
# remove missing data
CombOp = CombOp[completecases(CombOp), :]
# concatenate gene and alteration
CombOp[!, "GeneAlt"] = string.(CombOp[!, "GENE"], ":", CombOp[!, "ALTERATION"]);

In [ ]:
function GenConds(muts)
    Conds = DataFrame(Param = Any[], Val = Float64[])
        for i in 1:length(muts)        
            # check genes are in dictionary
            if haskey(param_mut, muts[i]) == true
                mut=muts[i]
                val=param_vals[mut]
                param=param_mut[mut]
                ParamVal = DataFrame(Param=param, Val=val)
                append!(Conds, ParamVal)
            else 
                continue
            end
        end
    return(Conds)
end

In [ ]:
function GenConds2(muts_count)
    p2C=[]
    mA=[]
    for i in 1:nrow(muts_count)
        param=muts_count[i,1]
        push!(p2C, param)
        val=muts_count[i,2]
        NUM=muts_count[i,3]
        PV=val^NUM
        push!(mA, PV)
    end
    return(p2C, mA)
end        

In [ ]:
# create mut dictionary
param_mut=Dict() 
# populate mut dictionary
for i in 1:nrow(muts2Params)
    GeneAlt = muts2Params[i,:GeneAlt]
    Param = muts2Params[i,:PARAMETER]
    param_mut[GeneAlt]=Param
end    

In [ ]:
# create gene:alt dictionary
gene_alt=Dict() 
# populate mut dictionary
for i in 1:nrow(muts2Params)
    GeneAlt = muts2Params[i,:GeneAlt]
    Param = muts2Params[i,:PARAMETER]
    gene_alt[Param]=GeneAlt
end    

In [ ]:
# create vals dictionary
param_vals=Dict()
# populate vals dictionary
for i in 1:nrow(muts2Params)
    GeneAlt = muts2Params[i,:GeneAlt]
    Val = muts2Params[i,:AMOUNT]
    param_vals[GeneAlt]=Val
end   

In [ ]:
# get data for individual patient
function getPatConds(patient)
    patient_df = subset(CombOp, :PATIENT => ByRow(==(patient)))
    
    # get mutations
    muts = Vector(patient_df[!, :GeneAlt])
    conds = GenConds(muts)
    muts_count = combine(groupby(conds, [:Param, :Val]), nrow => :count)
        
    ######################
    ### Set conditions ###
    ######################
    
    # set patient name
    patient = patient_df[1,1]
    
    conditions = ["all_mut"]
    
    # get conditions
    conds_run = GenConds2(muts_count)
    
    condsToPlot = Any["WT"]
    params = conds_run[1]
    values = conds_run[2]
    
    for i in 1:length(conds_run[1])
        folderName = string.(params[i], "_", values[i])
        push!(condsToPlot, folderName)
    end
    push!(condsToPlot, patient)
    
    GeneAlt = Any["WT"]
    for i in 1:length(params)
        label = gene_alt[params[i]]
        push!(GeneAlt, label)
    end
    push!(GeneAlt, "all_muts")
    
    return(condsToPlot, GeneAlt)
end

In [ ]:
condsToPlot1 = getPatConds("DLBCL-LS297")

In [ ]:
LS297_params = condsToPlot1[2]

In [ ]:
gene_alt[LS297_params[1]]

In [ ]:
function compareCellNumbersRelative(conditions,folderToPlot,num_mut)

    p3=plot(title="cell number - WT cell number",ylabel="cells > WT",labels=string.([1:maxGens]),xlim=(0,maxTimeTC),xticks = (0:60*24:maxTimeTC, string.(0:24:Int64(maxTimeTC))))
    p7=plot(title="cell number",ylabel="cells",labels=string.([1:maxGens]),xlim=(0,maxTimeTC),xticks = (0:60*24:maxTimeTC, string.(0:24:Int64(maxTimeTC))))
    WTorder=nothing
    condIndex=1
    WTyIndexes=[]
    thisCondScalar=0;
    WTArrayCellCountArray=[]
    synergyArray=[]
    
    if num_mut==1
        palettes=[[cA[3],cA[8]]]
    elseif num_mut==2
        palettes=[[cA[1],cA[8]],[cA[2],cA[8]],[cA[4],cA[8]],[cA[3],cA[8]]]
    elseif num_mut==3
        palettes=[[cA[1],cA[8]],[cA[2],cA[8]],[cA[4],cA[8]],[cA[5],cA[8]],[cA[3],cA[8]]]
    elseif num_mut==4
        palettes=[[cA[1],cA[8]],[cA[2],cA[8]],[cA[4],cA[8]],[cA[5],cA[8]],[cA[6],cA[8]],[cA[3],:gray]]
    elseif num_mut==5 
        palettes=[[cA[1],cA[8]],[cA[2],cA[8]],[cA[4],cA[8]],[cA[5],cA[8]],[cA[6],cA[8]],
                    [cA[7],cA[8]],[cA[3],cA[8]]]
    elseif num_mut==6 
        palettes=[[cA[1],cA[8]],[cA[2],cA[8]],[cA[4],cA[8]],[cA[5],cA[8]],[cA[6],cA[8]],
                    [cA[7],cA[8]],[cA[10],cA[8]],[cA[3],cA[8]]]        
    end
        
    for condToPlot in conditions

        thisColorPalette=palette(palettes[Int64(condIndex)],Int64(maxGens))
        crossSize=5;
        
        pathToPlot=folderToPlot*"/"*condToPlot*"/"

        #define some files to plot
        treeArrayFile=pathToPlot*"treeStructure.csv"
        cellCountFileName=pathToPlot*"cellCounts.csv"
        graphingArray=CSV.File(treeArrayFile,types=Float64, silencewarnings=true) |> Tables.matrix

        cellCountArray=zeros(maxGens,Int64(maxTimeTC))


        #first we need to sort by number of generations reached
        order=sortperm(graphingArray[:, end])
        if(condIndex==1)
            WTorder=order
            graphingArray=graphingArray[order, :]
        else
            graphingArray=graphingArray[WTorder, :]
        end

        nextYCoord=0
        #now we need to go through each
        for i in 1:size(graphingArray,1)
            thisCellMaxGen=Int64(graphingArray[i,end])
            cellsInThisCellMaxGen=2^(thisCellMaxGen)-1
            #nextYCoord=nextYCoord+cellsInThisCellMaxGen
            nextYCoord=nextYCoord+cellsInThisCellMaxGen
            middle=nextYCoord+1+cellsInThisCellMaxGen/2
            if(condIndex==1)
                append!(WTyIndexes,middle+10)
            else
                middle=WTyIndexes[i]
            end
            lastGenEnd=1
            nextGenMiddles=[]
            for thisGen in 1:thisCellMaxGen
                cellsInThisGen=2^(thisGen-1)
                stepSizePerCell=cellsInThisCellMaxGen/(cellsInThisGen-1)
                thisGenFateTime=graphingArray[i,thisGen]
                thisGenMiddles=copy(nextGenMiddles)
                cellCountArray[thisGen,Int64(round(lastGenEnd)):Int64(round(thisGenFateTime))-1]=cellCountArray[thisGen,Int64(round(lastGenEnd)):Int64(round(thisGenFateTime))-1].+cellsInThisGen
                lastGenEnd=graphingArray[i,thisGen]
            end  
            nextYCoord=nextYCoord+maximum([cellsInThisCellMaxGen,10])
        end
        originalCellCountArray=copy(cellCountArray)
        if condIndex==1
            WTArrayCellCountArray=copy(cellCountArray)
            #println(WTArrayCellCountArray)
            cellCountArray.=0
            synergyArray=zeros(length(conditions)-1,Int64(maxTimeTC))
        else
            cellCountArray=cellCountArray.-WTArrayCellCountArray
        end
        totalCountForPlotting=sum(originalCellCountArray,dims=1)
        totalCount=sum(cellCountArray,dims=1)
        if condIndex>1
           synergyArray[condIndex-1,:]=totalCount
        end
        generationArray=zeros(maxGens,size(cellCountArray,2))
        for thisGen in 1:maxGens
            thisGenCells=zeros(size(cellCountArray))
            thisGenCounts=cellCountArray[thisGen,:]
            for addingGen in thisGen:maxGens
                generationArray[addingGen,:]=generationArray[addingGen,:]+thisGenCounts
            end
            #comment out this line if you don't want individual generations to be plotted
            #plot!(p3,1:maxTimeTC,generationArray[thisGen,:][1:Int64(maxTimeTC)],linecolor = thisColorPalette[thisGen],label=false)
        end
        plot!(p3,[1:maxTimeTC],totalCount',linecolor =thisColorPalette[1], linealpha=0.75 ,label=condToPlot,linewidth=3)
        plot!(p7,[1:maxTimeTC],totalCountForPlotting',linecolor =thisColorPalette[1], linealpha=0.75,label=condToPlot,linewidth=3)

        
        #println(totalCount)
        #println(condToPlot)
        plot!(p3,legend = true,size=(900,400),layout = grid(3, 1,heights=[0.8,0.1,0.1]))
        #display(thisPlot)
        plot!(p7,legend = true,size=(900,400),layout = grid(3, 1,heights=[0.8,0.1,0.1]))

        condIndex=condIndex+1
        
    end
    
    expectedIfAdditive=sum(synergyArray[1:end-1,:],dims=1)

    actualResult=synergyArray[end,:]'

    lower=minimum(vcat(actualResult,expectedIfAdditive),dims=1)
    upper=maximum(vcat(actualResult,expectedIfAdditive),dims=1)
    actualResult=actualResult'
    lower=lower'
    upper=upper'
    p5=plot([1:maxTimeTC], [actualResult actualResult], fillrange=[lower upper], fillalpha=0.35, c = [:red :blue], label = ["> additive (synergistic)" "<= additive"], legend = :topleft)
    plot!(p5,size=(900,400),layout = grid(3, 1,heights=[0.8,0.1,0.1]),title="expected if additive vs actual combo",ylabel="cell number over WT",labels=string.([1:maxGens]),xlim=(0,maxTimeTC),xticks = (0:60*24:maxTimeTC, string.(0:24:Int64(maxTimeTC))))
    differenceFromExpected=actualResult.-expectedIfAdditive
    p6=plot([1:maxTimeTC], differenceFromExpected)
    plot!(p6,size=(900,400),layout = grid(3, 1,heights=[0.8,0.1,0.1]),title="[actual combo] - [additive prediction]",ylabel="cell number over WT",xlabel="time (h)",labels=string.([1:maxGens]),xlim=(0,maxTimeTC),xticks = (0:60*24:maxTimeTC, string.(0:24:Int64(maxTimeTC))),legend=true)
    thisPlot=plot(p7,size=(900,600),layout = grid(2, 1,heights=[0.5,0.5]))
    display(thisPlot)
end

In [ ]:
function compareCellNumbersRelative2(patient)

    p3=plot(title="cell number - WT cell number",ylabel="cells > WT",labels=string.([1:maxGens]),xlim=(0,maxTimeTC),xticks = (0:60*24:maxTimeTC, string.(0:24:Int64(maxTimeTC))))
    p7=plot(title="cell number",ylabel="cells",labels=string.([1:maxGens]),xlim=(0,maxTimeTC),xticks = (0:60*24:maxTimeTC, string.(0:24:Int64(maxTimeTC))))
    WTorder=nothing
    condIndex=1
    WTyIndexes=[]
    thisCondScalar=0;
    WTArrayCellCountArray=[]
    synergyArray=[]
    
    condsToPlot = getPatConds(patient)[1]
    labels = getPatConds(patient)[2]
    num_mut = length(condsToPlot)-2
    
    if num_mut==1
        palettes=[[cA[3],cA[8]]]
    elseif num_mut==2
        palettes=[[cA[1],cA[8]],[cA[2],cA[8]],[cA[4],cA[8]],[cA[3],cA[8]]]
    elseif num_mut==3
        palettes=[[cA[1],cA[8]],[cA[2],cA[8]],[cA[4],cA[8]],[cA[5],cA[8]],[cA[3],cA[8]]]
    elseif num_mut==4
        palettes=[[cA[1],cA[8]],[cA[2],cA[8]],[cA[4],cA[8]],[cA[5],cA[8]],[cA[6],cA[8]],[cA[3],:gray]]
    elseif num_mut==5 
        palettes=[[cA[1],cA[8]],[cA[2],cA[8]],[cA[4],cA[8]],[cA[5],cA[8]],[cA[6],cA[8]],
                    [cA[7],cA[8]],[cA[3],cA[8]]]
    elseif num_mut==6 
        palettes=[[cA[1],cA[8]],[cA[2],cA[8]],[cA[4],cA[8]],[cA[5],cA[8]],[cA[6],cA[8]],
                    [cA[7],cA[8]],[cA[10],cA[8]],[cA[3],cA[8]]]        
    end
        
    for condToPlot in 1:length(condsToPlot)

        thisColorPalette=palette(palettes[Int64(condIndex)],Int64(maxGens))
        crossSize=5;
        
        if condsToPlot[condToPlot]=="WT"
            pathToPlot="SSDoutputs/allMutSims/WT/"
        elseif condsToPlot[condToPlot]==patient
            pathToPlot="SSDoutputs/allMutSims/"*patient*"/all_mut/"
        else pathToPlot="SSDoutputs/allMutSims/"*condsToPlot[condToPlot]*"/mut/"
        end
            
        #define some files to plot
        treeArrayFile=pathToPlot*"treeStructure.csv"
        cellCountFileName=pathToPlot*"cellCounts.csv"
        graphingArray=CSV.File(treeArrayFile,types=Float64, silencewarnings=true) |> Tables.matrix

        cellCountArray=zeros(maxGens,Int64(maxTimeTC))


        #first we need to sort by number of generations reached
        order=sortperm(graphingArray[:, end])
        if(condIndex==1)
            WTorder=order
            graphingArray=graphingArray[order, :]
        else
            graphingArray=graphingArray[WTorder, :]
        end

        nextYCoord=0
        #now we need to go through each
        for i in 1:size(graphingArray,1)
            thisCellMaxGen=Int64(graphingArray[i,end])
            cellsInThisCellMaxGen=2^(thisCellMaxGen)-1
            nextYCoord=nextYCoord+cellsInThisCellMaxGen
            middle=nextYCoord+1+cellsInThisCellMaxGen/2
            if(condIndex==1)
                append!(WTyIndexes,middle+10)
            else
                middle=WTyIndexes[i]
            end
            lastGenEnd=1
            nextGenMiddles=[]
            for thisGen in 1:thisCellMaxGen
                cellsInThisGen=2^(thisGen-1)
                stepSizePerCell=cellsInThisCellMaxGen/(cellsInThisGen-1)
                thisGenFateTime=graphingArray[i,thisGen]
                thisGenMiddles=copy(nextGenMiddles)
                cellCountArray[thisGen,Int64(round(lastGenEnd)):Int64(round(thisGenFateTime))-1]=cellCountArray[thisGen,Int64(round(lastGenEnd)):Int64(round(thisGenFateTime))-1].+cellsInThisGen
                lastGenEnd=graphingArray[i,thisGen]
            end  
            nextYCoord=nextYCoord+maximum([cellsInThisCellMaxGen,10])
        end
        originalCellCountArray=copy(cellCountArray)
        if condIndex==1
            WTArrayCellCountArray=copy(cellCountArray)
            #println(WTArrayCellCountArray)
            cellCountArray.=0
            synergyArray=zeros(length(condsToPlot)-1,Int64(maxTimeTC))
        else
            cellCountArray=cellCountArray.-WTArrayCellCountArray
        end
        totalCountForPlotting=sum(originalCellCountArray,dims=1)
        totalCount=sum(cellCountArray,dims=1)
        if condIndex>1
           synergyArray[condIndex-1,:]=totalCount
        end
        generationArray=zeros(maxGens,size(cellCountArray,2))
        for thisGen in 1:maxGens
            thisGenCells=zeros(size(cellCountArray))
            thisGenCounts=cellCountArray[thisGen,:]
            for addingGen in thisGen:maxGens
                generationArray[addingGen,:]=generationArray[addingGen,:]+thisGenCounts
            end
            #comment out this line if you don't want individual generations to be plotted
            #plot!(p3,1:maxTimeTC,generationArray[thisGen,:][1:Int64(maxTimeTC)],linecolor = thisColorPalette[thisGen],label=false)
        end
        plot!(p3,[1:maxTimeTC],totalCount',linecolor =thisColorPalette[1], linealpha=0.75 ,label=labels[condToPlot],linewidth=3)
        plot!(p7,[1:maxTimeTC],totalCountForPlotting',linecolor =thisColorPalette[1], linealpha=0.75,label=labels[condToPlot],linewidth=3)

        
        #println(totalCount)
        #println(condToPlot)
        plot!(p3,legend = true,size=(900,400),layout = grid(3, 1,heights=[0.8,0.1,0.1]))
        #display(thisPlot)
        plot!(p7,legend = true,size=(900,400),layout = grid(3, 1,heights=[0.8,0.1,0.1]))

        condIndex=condIndex+1
        
    end
    
    expectedIfAdditive=sum(synergyArray[1:end-1,:],dims=1)

    actualResult=synergyArray[end,:]'

    lower=minimum(vcat(actualResult,expectedIfAdditive),dims=1)
    upper=maximum(vcat(actualResult,expectedIfAdditive),dims=1)
    actualResult=actualResult'
    lower=lower'
    upper=upper'
    p5=plot([1:maxTimeTC], [actualResult actualResult], fillrange=[lower upper], fillalpha=0.35, c = [:red :blue], label = ["> additive (synergistic)" "<= additive"], legend = :topleft)
    plot!(p5,size=(900,400),layout = grid(3, 1,heights=[0.8,0.1,0.1]),title="expected if additive vs actual combo",ylabel="cell number over WT",labels=string.([1:maxGens]),xlim=(0,maxTimeTC),xticks = (0:60*24:maxTimeTC, string.(0:24:Int64(maxTimeTC))))
    differenceFromExpected=actualResult.-expectedIfAdditive
    p6=plot([1:maxTimeTC], differenceFromExpected)
    plot!(p6,size=(900,400),layout = grid(3, 1,heights=[0.8,0.1,0.1]),title="[actual combo] - [additive prediction]",ylabel="cell number over WT",xlabel="time (h)",labels=string.([1:maxGens]),xlim=(0,maxTimeTC),xticks = (0:60*24:maxTimeTC, string.(0:24:Int64(maxTimeTC))),legend=true)
    thisPlot=plot(p7,p5,size=(900,600),layout = grid(2, 1,heights=[0.5,0.5]))
    display(thisPlot)
end

In [ ]:
DLBCL_LS297 = getPatConds("DLBCL-LS297")

In [ ]:
## import patient list
patients_withOS = DataFrame(CSV.File("patient_OS_data.csv"))
patients_withOS = patients_withOS[!, :patient_ID]

In [ ]:
DLBCL_LS1546[1][1]

In [ ]:
DLBCL_LS1546[1]

In [ ]:
param_mut

In [ ]:
folderToPlot = "SSDoutputs/DHx5_constantIKK"
conditions=["WT", "Myc_x5", "Bcl2_x5", "double hit"]
compareCellNumbersRelative(conditions,folderToPlot,2)

In [ ]:
folderToPlot = "SSDoutputs/allMutSims/first_run/DLBCL-DFCI_DLBCL_GOE07"
conditions=["all_mut"]
compareCellNumbersRelative(conditions,folderToPlot,1)

In [ ]:
function CellNumbers(patient)
                
        pathToPlot="SSDoutputs/allMutSims/"*patient*"/all_mut/"
    
        WTorder=nothing
        WTyIndexes=[]
        thisCondScalar=0;
        WTArrayCellCountArray=[]
        synergyArray=[]

        #define some files to plot
        treeArrayFile=pathToPlot*"treeStructure.csv"
        cellCountFileName=pathToPlot*"cellCounts.csv"
        graphingArray=CSV.File(treeArrayFile,types=Float64, silencewarnings=true) |> Tables.matrix

        cellCountArray=zeros(maxGens,Int64(maxTimeTC))

        #first we need to sort by number of generations reached
        order=sortperm(graphingArray[:, end])
        
        WTorder=order
        graphingArray=graphingArray[order, :]
        
        nextYCoord=0
        #now we need to go through each
        for i in 1:size(graphingArray,1)
            thisCellMaxGen=Int64(graphingArray[i,end])
            cellsInThisCellMaxGen=2^(thisCellMaxGen)-1
            #nextYCoord=nextYCoord+cellsInThisCellMaxGen
            nextYCoord=nextYCoord+cellsInThisCellMaxGen
            middle=nextYCoord+1+cellsInThisCellMaxGen/2
            append!(WTyIndexes,middle+10)

            lastGenEnd=1
            nextGenMiddles=[]
            for thisGen in 1:thisCellMaxGen
                cellsInThisGen=2^(thisGen-1)
                stepSizePerCell=cellsInThisCellMaxGen/(cellsInThisGen-1)
                thisGenFateTime=graphingArray[i,thisGen]
                thisGenMiddles=copy(nextGenMiddles)
                cellCountArray[thisGen,Int64(round(lastGenEnd)):Int64(round(thisGenFateTime))-1]=cellCountArray[thisGen,Int64(round(lastGenEnd)):Int64(round(thisGenFateTime))-1].+cellsInThisGen
                lastGenEnd=graphingArray[i,thisGen]
            end  
            nextYCoord=nextYCoord+maximum([cellsInThisCellMaxGen,10])
        end
        originalCellCountArray=copy(cellCountArray)
        WTArrayCellCountArray=copy(cellCountArray)
        #println(WTArrayCellCountArray)
        cellCountArray.=0
        synergyArray=zeros(0,Int64(maxTimeTC))

        totalCountForPlotting=sum(originalCellCountArray,dims=1)
        #totalCount=sum(cellCountArray,dims=1)
        maxCells = maximum(totalCountForPlotting)
        return(maxCells)
end

In [ ]:
patient_OS = DataFrame(CSV.File("patient_OS_data.csv"))

In [ ]:
patient_status = DataFrame(CSV.File("patient_status_data.csv"))

In [ ]:
patient_OS_stat = innerjoin(patient_OS, patient_status, on = :patient_ID)

In [ ]:
patientSims = readdir("SSDoutputs/allMutSims/")
setdiff!(patientSims, ["first_run"])

In [ ]:
patientSimsOS = patient_OS_stat[in.(patient_OS_stat[!, :patient_ID], [patientSims]),:]

In [ ]:
patients_OS_CN = DataFrame(OS=Int64[], patient_ID=Any[], Status=Any[], CN=Int64[])

In [ ]:
for i in 1:nrow(patientSimsOS)
    patOSdf = DataFrame(patientSimsOS[i, 1:3])
    patient = patOSdf[:, :patient_ID]
    CN = CellNumbers(patient[1])
    patOSdf[!, :CN] = [CN]
    append!(patients_OS_CN, patOSdf)
end    

In [ ]:
patients_OS_CN

In [ ]:
sort!(patients_OS_CN, :OS)

In [ ]:
CSV.write("lowOS_res.csv", patients_OS_CN)

In [ ]:
using Pkg
Pkg.add("StatsPlots") 
using StatsPlots, KernelDensity

In [ ]:
dens = kde((patients_OS_CN[:, :OS],patients_OS_CN[:, :CN]))
plot(dens)

In [ ]:
patients_OS_CN

In [ ]:
patients_OS_CN = subset(patients_OS_CN, :CN => ByRow(>(0)))

In [ ]:
plot(patients_OS_CN[:, :OS], patients_OS_CN[:, :CN], 
        seriestype=:scatter, 
        ylab="peak cell number", 
        xlab="OS",
        group=patients_OS_CN[:, :Status],
        labels = ["living" "deceased"],
        ylim=(0, 3e3),
        xlim=(0, 160))

In [ ]:
lowOS_res = subset(patients_OS_CN, :OS => ByRow(<=(35)))

In [ ]:
### run all code together
patientSims = readdir("SSDoutputs/allMutSims/")
setdiff!(patientSims, ["first_run"])
patientSimsOS = patient_OS_stat[in.(patient_OS_stat[!, :patient_ID], [patientSims]),:]
patients_OS_CN = DataFrame(OS=Int64[], patient_ID=Any[], Status=Any[], CN=Int64[])
for i in 1:nrow(patientSimsOS)
    patOSdf = DataFrame(patientSimsOS[i, 1:3])
    patient = patOSdf[:, :patient_ID]
    CN = CellNumbers(patient[1])
    patOSdf[!, :CN] = [CN]
    append!(patients_OS_CN, patOSdf)
end    
plot(patients_OS_CN[:, :OS], patients_OS_CN[:, :CN], 
        seriestype=:scatter, 
        ylab="peak cell number", 
        xlab="OS",
        group=patients_OS_CN[:, :Status],
        labels = ["living" "deceased"],
       )

In [ ]:
CSV.write("OS_CN_res.csv", patients_OS_CN)

In [ ]:
patients_OS_CN

In [ ]:
CSV.write("patients_OS_CN", patients_OS_CN)

In [ ]:
lowOS_res = subset(lowOS_res, :CN => ByRow(>(0)))

In [ ]:
sort!(lowOS_res, :OS)

In [ ]:
CSV.write("lowOS_res.csv", patients_OS_CN)

In [ ]:
model = lm(@formula(y ~ x), df)

In [ ]:
plot(lowOS_res[:, :OS], lowOS_res[:, :CN], 
        seriestype=:scatter, 
        ylab="peak cell number", 
        xlab="OS",
        legend=false,
        smooth=true)

In [ ]:
patient_CoO = DataFrame(CSV.File("patient_CoO.csv"))

In [ ]:
patientsSims2 = patients_OS_CN[!, :patient_ID]

In [ ]:
patientSims_CoO = patient_CoO[in.(patient_CoO[!, :patient_ID], [patientsSims2]),:]

In [ ]:
patient_OSCNCoO = innerjoin(patients_OS_CN, patientSims_CoO, on = :patient_ID)

In [ ]:
patient_ABCGC = subset(patient_OSCNCoO, :CoO => ByRow(!=("DLBCLNOS")))

In [ ]:
plot(patient_OSCNCoO[:, :OS], patient_OSCNCoO[:, :CN], 
        seriestype=:scatter, 
        ylab="peak cell number", 
        xlab="OS",
        group=patient_OSCNCoO[:, :CoO],
        ylim=(0, 3e3),
        xlim=(0, 160),
        size=(700,500),
        smooth=true)

In [ ]:
plot(patient_ABCGC[:, :OS], patient_ABCGC[:, :CN], 
        seriestype=:scatter, 
        ylab="peak cell number", 
        xlab="OS",
        group=patient_ABCGC[:, :CoO],
        ylim=(0, 3e3),
        xlim=(0, 160),
        size=(700,500),
        smooth=true)

In [ ]:
sort(patients_OS_CN, :OS)

In [ ]:
CSV.write("patients_OS_CN.csv", patients_OS_CN)

In [ ]:
patientswithOS = patients_OS_CN[!, :patient_ID]

In [ ]:
patients_noOS = setdiff!(patientSims, patientswithOS)
show(stdout, "text/plain", patients_noOS)

In [ ]:
function plotPat_varIKK(patient_name)
    
    # get data for individual patient
    patient_df = subset(patient_data, :PATIENT => ByRow(==(patient_name))) 
    
    # get genes
    genes = Vector(patient_df[!, :GENE])
    
    # set conditions
    n = nrow(patient_df)
    
    if n==2
            conditions= ["WT", patient_df[!, :GENE][1], patient_df[!, :GENE][2], "all_mut"]
        elseif n==3 
            conditions= ["WT", patient_df[!, :GENE][1], patient_df[!, :GENE][2],
                        patient_df[!, :GENE][3], "all_mut"]
        elseif n==4 
            conditions= ["WT", patient_df[!, :GENE][1], patient_df[!, :GENE][2],
                        patient_df[!, :GENE][3], patient_df[!, :GENE][4], "all_mut"]
        elseif n==5 
            conditions= ["WT", patient_df[!, :GENE][1], patient_df[!, :GENE][2],
                        patient_df[!, :GENE][3], patient_df[!, :GENE][4],
                        patient_df[!, :GENE][5], "all_mut"]
        elseif n==6 
            conditions= ["WT", patient_df[!, :GENE][1], patient_df[!, :GENE][2],
                        patient_df[!, :GENE][3], patient_df[!, :GENE][4],
                        patient_df[!, :GENE][5], patient_df[!, :GENE][6], "all_mut"]
    end
    
    folderToPlot = "SSDoutputs/DLBCL_patient_modelling/loopSims/varIKK/"*patient_name
    
    compareCellNumbersRelative(conditions,folderToPlot,n)
end

In [ ]:
folderToPlot = "SSDoutputs/DLBCL_patient_modelling/paper/DLBCL-LS2599"
conditions=["WT", "PARP1", "MCL1", "CCNB1", "CYCS", "PRDM1", "all_mut"]
compareCellNumbersRelative(conditions,folderToPlot,5)

In [ ]:
folderToPlot = "SSDoutputs/DLBCL_patient_modelling/paper/DLBCL-LS2599inhib"
conditions=["WT", "noPARP1", "noMCL1", "noCCNB1", "noCYCS", "noPRDM1", "all_mut"]
compareCellNumbersRelative(conditions,folderToPlot,5)

In [ ]:
folderToPlot = "SSDoutputs/DLBCL_patient_modelling/DLBCL-LS148incmut"
conditions=["WT", "MCL1-BCL2", "PARP1", "all_mut"]
compareCellNumbersRelative(conditions,folderToPlot,2)

In [ ]:
folderToPlot = "SSDoutputs/DLBCL_patient_modelling/DLBCL-LS4619incmut"
conditions=["WT", "REL", "PRDM1", "all_mut"]
compareCellNumbersRelative(conditions,folderToPlot,2)

In [ ]:
folderToPlot = "SSDoutputs/DLBCL_patient_modelling/DLBCL-LS1546incmut"
conditions=["WT", "BCL2", "PRDM1", "CDKN1B", "PAX5_KO", "all_mut"]
compareCellNumbersRelative(conditions,folderToPlot,4)

In [ ]:
folderToPlot = "SSDoutputs/DLBCL_patient_modelling/DLBCL-RICOVER_496incmut"
conditions=["WT", "PARP1", "MCL1", "CASP3", "MYC++", "all_mut"]
compareCellNumbersRelative(conditions,folderToPlot,4)

In [ ]:
folderToPlot = "SSDoutputs/DLBCL_patient_modelling/DLBCL-RICOVER_1106incmut"
conditions=["WT", "BAX", "BCL2", "CCNE1", "NFKBIB", "IRF8+", "all_mut"]
compareCellNumbersRelative(conditions,folderToPlot,5)

In [ ]:
folderToPlot = "SSDoutputs/DLBCL_patient_modelling/DLBCL-RICOVER_1169incmut"
conditions=["WT", "MCL1", "PARP1", "REL", "BCORmut", "all_mut"]
compareCellNumbersRelative(conditions,folderToPlot,4)

In [ ]:
plotPat_varIKK("DLBCL-RICOVER_1263")

In [ ]:
plotPat_varIKK("DLBCL-LS2599")